# ABS Monthly Labour Force 6202

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Headline-charts" data-toc-modified-id="Headline-charts-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Headline charts</a></span></li><li><span><a href="#State-comparisons" data-toc-modified-id="State-comparisons-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>State comparisons</a></span></li><li><span><a href="#National/State-Growth" data-toc-modified-id="National/State-Growth-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>National/State Growth</a></span></li><li><span><a href="#Gendered-comparisons" data-toc-modified-id="Gendered-comparisons-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Gendered comparisons</a></span></li><li><span><a href="#Victorian/COVID19-comparisons" data-toc-modified-id="Victorian/COVID19-comparisons-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Victorian/COVID19 comparisons</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
from pathlib import Path
import calendar
import datetime

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from python_env import python_env
from finalise_plot import finalise_plot
from abs_common import (get_fs_constants,
                        get_plot_constants,
                        get_ABS_meta_and_data,
                        get_identifier,
                        plot_Mgrowth)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
cat_num = '6202'
CACHE_DIR, CHART_DIR, source = get_fs_constants(cat_num)

## Python environment

In [3]:
python_env()

--------------------------------------------------
System:     Darwin
Release:    20.2.0
Machine:    x86_64
Processor:  i386
RAM:        16GB
--------------------------------------------------
Python:     3.8.3
Psutil:     5.7.2
Pandas:     1.1.5
Numpy:      1.19.2
Matplotlib: 3.3.2
--------------------------------------------------


## Get data from ABS

In [4]:
# do the data capture and extraction to a DataFrame
url_template = ('https://www.abs.gov.au/statistics/labour/'
                'employment-and-unemployment/labour-force-australia/'
                'MONTH-YEAR/6202_all%20spreadsheet.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

File has been cached already
File for nov-2020 of size 41.8 MB


## Plot

### Set-up some constants we will use across all plots

In [5]:
RECENT, plot_times, plot_frames, plot_tags = get_plot_constants(data)

# reference point for COVID comparisons
REFERENCE = pd.Timestamp('2020-02-01')
REF_STRING = f'{calendar.month_abbr[REFERENCE.month]} {REFERENCE.year}'
index_base = 100

### Headline charts

Note: with COVID, the ABS has stopped publishing the Trend series.  

In [6]:
headlines = {
    # data item                                       # table number
    'Employed total ;  Persons ;':                    '1',
    '> Employed part-time ;  Persons ;':              '1',
    '> Employed full-time ;  Persons ;':              '1',
    'Unemployed total ;  Persons ;':                  '1',
    'Unemployment rate ;  Persons ;':                 '1', 
    'Participation rate ;  Persons ;':                '1',
    'Labour force total ;  Persons ;':                '1',
    'Monthly hours worked in all jobs ;  Persons ;':  '19',
}
series_type = "Seasonally Adjusted"

for headline, table in headlines.items():

    seasadj, units = get_identifier(meta, headline, 
                                    series_type, table)
    
    for df, tag in zip(plot_frames, plot_tags):

        ax = df[[seasadj]].plot(c='#dd0000', lw=2)
        ax.legend([series_type], loc='best')
        title = f'Australia: {headline}'
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source} {table}',
                     )

### State comparisons 

In [7]:
states = {
    # state                     # table
    "NSW":                      '4',
    "Vic":                      '5',
    "Qld":                      "6",
    "SA":                       "7",
    "WA":                       "8",
    "Tas":                      "9",
}

plots = [
    'Unemployment rate ;  Persons ;', 
    'Participation rate ;  Persons ;',
]

colours = ['blue', 'darkorchid', 'brown', 'olivedrab', 'red', 
           'darkorange', 'darkgoldenrod', 'teal',]

series_type = "Seasonally Adjusted"

for plot in plots:
    
    collection = []
    tables = []
    for state, table in states.items():
        id, units = get_identifier(meta, plot, series_type, table)
        collection.append(id)
        tables.append(table)
        
    for frame, tag in zip(plot_frames, plot_tags):
        ax = frame[collection].plot(lw=2, color=colours)
        ax.legend(states.keys(), loc='best')
        title = f'Australian states: {plot}'
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source}s {",".join(tables)}',
                      lfooter=f'From {series_type.lower()} series')

### National/State Growth

In [8]:
series_type = "Seasonally Adjusted"
tables = ['12', '12a']
plot_stems = [
    'Employed total ;  Persons ;',
    'Unemployed total ;  Persons ;',
    'Labour force total ;  Persons ;',
]

for stem in plot_stems:
    for table in tables:
        
        descriptions = meta[
            (meta['Table'] == table) &
            (meta['Series Type'] == series_type) &
            (meta['Data Item Description'].str.contains(stem))
        ]['Data Item Description']
        
        for desc in descriptions:
            series, units = get_identifier(meta, desc, 
                                           series_type, 
                                           table)
            for plot_from, plot_tag in zip(plot_times, 
                                           plot_tags):
                title = f'Growth in {desc}'
                plot_Mgrowth(data[series], 
                             title, plot_from, 
                             plot_tag, CHART_DIR,
                             lfooter=f'Calculated from {series_type.lower()} series', 
                             rfooter=f'{source} {table}',
                )

### Gendered comparisons

In [9]:
genders = ['Males', 'Females']
colours = ['cornflowerblue', 'hotpink']

TOKEN = 'by gender' # note: also appears in plot title
plots = {
    # data description                       # table
    f'Employed total ;  > {TOKEN} ;':       '1',
    f'> Employed part-time ;  > {TOKEN} ;': '1',
    f'> Employed full-time ;  > {TOKEN} ;': '1',
    f'Unemployed total ;  > {TOKEN} ;':     '1',
    f'Labour force total ;  > {TOKEN} ;':     '1',
    f"Unemployment rate ;  > {TOKEN} ;":    '1',
    f"Participation rate ;  > {TOKEN} ;":   '1',
    f'Monthly hours worked in all jobs ;  > {TOKEN} ;':  '19',
}

series_type = "Seasonally Adjusted"

for plot, table in plots.items():
    
    # assemble data
    collection = []
    for gender in genders:
        this_gender = plot.replace(TOKEN, gender)
        ident, units = get_identifier(meta, this_gender, 
                                   series_type, table)
        collection.append(ident)

    # plot
    for frame, tag in zip(plot_frames, plot_tags):
        ax = frame[collection].plot(lw=2, color=colours)
        ax.legend(genders, loc='best')
        title = f"Australia: {plot}"
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source} {table}',
                      lfooter=f'From {series_type.lower()} series')
        
        # COVID - plot index refereced to Feb 2020
        if '000' in units:
            # index comparison
            selection = frame[collection].copy()
            reference_row = selection.loc[REFERENCE]
            selection = selection.div(reference_row) * index_base
            ax = selection.plot(lw=2, color=colours)
            ax.legend(genders, loc='best')
            title = f"Australia: {plot} (Index)"
            finalise_plot(ax, title, 
                  f'Index ({REF_STRING} = {index_base})', 
                  f'{tag}', CHART_DIR,
                  rfooter=f'{source} {table}',
                  lfooter=f'Calculated from {series_type.lower()} series')              

### Victorian/COVID19 comparisons

In [10]:
table = '12'
plots = [
    'Employed total ;  Persons ;  X ;',
    '> Employed full-time ;  Persons ;  X ;',
    'Unemployed total ;  Persons ;  X ;',
    'Labour force total ;  Persons ;  X ;',
]
df = data[data.index >= RECENT]
series_type = "Seasonally Adjusted"

for plot in plots:
    # get data - calculate index
    national, _ = get_identifier(meta, 
                    plot.replace('X', 'Australia'), 
                    series_type, table)
    victoria, _ = get_identifier(meta, 
                    plot.replace('X', '> Victoria'), 
                    series_type, table)
    frame = df[[victoria, national]].copy()
    frame[national] = frame[national] - frame[victoria] # ROA
    reference_row = frame.loc[REFERENCE]
    frame = frame.div(reference_row) * index_base

    # plot 
    ax = frame.plot(lw=2, color=['darkorange', 'blue'])
    ax.legend(['Victoria', 'Rest of Australia'], loc='best')
    title = f"Vic v Aus: {plot.replace('X ;', 'Index')}"
    finalise_plot(ax, title, 
                  f'Index ({REF_STRING} = {index_base})', 
                  'COVID-Index', CHART_DIR,
                  rfooter=f'{source} {table}',
                  lfooter=f'Calculated from {series_type.lower()} series')      

## Finished

In [11]:
print('Finished')

Finished
